In [34]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

imagen_bgr = cv2.imread('reloje.png')
imagen_rgb = cv2.cvtColor(imagen_bgr, cv2.COLOR_BGR2RGB)

gris = cv2.cvtColor(imagen_bgr, cv2.COLOR_BGR2GRAY)
gris = cv2.GaussianBlur(gris, (5, 5), 1)
gris = cv2.equalizeHist(gris)

bordes = cv2.Canny(gris, 50, 150)

lineas = cv2.HoughLinesP(bordes, 1, np.pi/180, threshold=80, minLineLength=50, maxLineGap=10)

centro_x = imagen_bgr.shape[1] // 2
centro_y = imagen_bgr.shape[0] // 2

def calcular_angulo(x1, y1, x2, y2):
    dist1 = np.hypot(x1 - centro_x, y1 - centro_y)
    dist2 = np.hypot(x2 - centro_x, y2 - centro_y)
    if dist1 > dist2:
        dx = x1 - centro_x
        dy = centro_y - y1
    else:
        dx = x2 - centro_x
        dy = centro_y - y2
    angulo = np.degrees(np.arctan2(dy, dx))
    angulo = (90 - angulo) % 360
    return angulo

def invertir_linea(p1, p2, cx, cy):
    dist1 = np.hypot(p1[0] - cx, p1[1] - cy)
    dist2 = np.hypot(p2[0] - cx, p2[1] - cy)
    if dist1 > dist2:
        extremo = p1
    else:
        extremo = p2
    invertido = (2 * cx - extremo[0], 2 * cy - extremo[1])
    return (cx, cy), invertido

manecillas = []
if lineas is not None:
    for linea in lineas:
        x1, y1, x2, y2 = linea[0]
        longitud = np.hypot(x2 - x1, y2 - y1)
        distancia_centro = min(np.hypot(x1 - centro_x, y1 - centro_y), np.hypot(x2 - centro_x, y2 - centro_y))
        if distancia_centro < 100 and 40 < longitud < 300:
            angulo = calcular_angulo(x1, y1, x2, y2)
            manecillas.append({
                'puntos': (x1, y1, x2, y2),
                'longitud': longitud,
                'angulo': angulo
            })

if len(manecillas) >= 3:
    manecillas.sort(key=lambda x: x['longitud'])
    horario = manecillas[0]    
    minutero = manecillas[-2]  
    segundero = manecillas[-1]

    hora = int((horario['angulo'] / 30) % 12)
    fraccion_hora = (horario['angulo'] / 30) % 12
    if abs(fraccion_hora - hora) > 0.5:
        hora = (hora + 1) % 12
    p_ini, p_fin = invertir_linea(
        (minutero['puntos'][0], minutero['puntos'][1]),
        (minutero['puntos'][2], minutero['puntos'][3]),
        centro_x, centro_y
    )
    angulo_minuto = calcular_angulo(p_ini[0], p_ini[1], p_fin[0], p_fin[1])
    minutos = int((angulo_minuto / 6) % 60)
    segundos = int((segundero['angulo'] / 6) % 60) 
    imagen_resultado = imagen_rgb.copy()
    cv2.line(imagen_resultado, (horario['puntos'][0], horario['puntos'][1]), (horario['puntos'][2], horario['puntos'][3]),(0, 0, 255), 5)
    cv2.line(imagen_resultado, p_ini, p_fin, (0, 255, 0), 5)
    cv2.line(imagen_resultado,(segundero['puntos'][0], segundero['puntos'][1]),(segundero['puntos'][2], segundero['puntos'][3]), (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(imagen_resultado)
    plt.title(f'Hora: {hora}:{minutos:02d}:{segundos:02d}')
    plt.axis('off')
    plt.show()
